In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM  , AutoTokenizer
import torch


# now we will use some questions as our data



data =[
    {
        'question': "What was Liana Barrientos charged with?",
        'answer': "Liana Barrientos is charged with two counts of 'offering a false instrument for filing in the first degree'."
    },
    {
        'question': "How many times has Liana Barrientos been married?",
        'answer': "Liana Barrientos has been married 10 times, with nine marriages occurring between 1999 and 2002."
    },
    {
        'question': "Is Liana Barrientos still married?",
        'answer': "It is believed that Liana Barrientos is still married to four men."
    },
    # Add more question-answer pairs relevant to your dataset
      ]


## load model and tokenizer


model =  AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

In [10]:
inputs = tokenizer([dataset['question'] for dataset in data], truncation=True, padding=True, return_tensors='pt')
labels = tokenizer([dataset['answer'] for dataset in data], truncation=True, padding=True, return_tensors='pt')

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
decoder_input_ids = labels['input_ids']
decoder_attention_mask = labels['attention_mask']

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 5




import torch.nn as nn
loss_function = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    decoder_input_ids = decoder_input_ids.to(device)
    decoder_attention_mask = decoder_attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask)
    logits = outputs.logits
    loss = loss_function(logits.view(-1, logits.shape[-1]), decoder_input_ids.view(-1))
    loss.backward()
    optimizer.step()

In [18]:
import warnings
warnings.filterwarnings('ignore')



q1 = 'What was Liana Barrientos charged with?'

inp_ids = tokenizer.encode(q1 , truncation=True, padding =True , return_tensors='pt').to(device)
response  = model.generate(inp_ids)

print(f'response: {tokenizer.decode(response[0] ,skip_special_tokens=True )}')


response: What was Liana Barrientos charged with? Liana barrientrientos. Liana is the daughter of former first lady of Venezuela, Liana L. Barrientes. L. is the wife of former president L. Hugo Barrientez. L is the mother of L.
